# Geometric transformations

Now we know that every pixel has two properties:

1. Coordinate of the pixel in an image grid (row, column) or (y, x).
2. Intensity of the pixel (either one value or vector of values).

In this class we will discuss transformation of coordinates of the pixels. Image transformation implies producing a new image based on the current one. Python makes it easy to perform basic transformations on digital images either manually, by doing manipulations with arrays, or by using methods from dedicated libraries.

Some common geometric operations include:    

*  Cropping, flipping, merging
*  Resizing/subsampling
*  Rotation 
*  Shearing

There are also more advance transformations, e.g. projective which we will not touch upon. In addition to that you can unfold and refold images to data matrix which we will briefly discuss.


## Image transformations by arrays manipulations

Let's start with cropping. *Cropping* involves removing parts of the image to focus on a particular area of interest, often used for removing unwanted elements in the image. Cropping is easy to do by subsetting rows and columns of the array with color intensities.

Let's load the apple image again and convert it to NumPy array:

In [ ]:
import numpy as np
from PIL import Image

img = Image.open("datasets/Apples.jpg")
img_arr = np.array(img)
img_arr.shape

Let's split the image to two, so the first one contains only the red apples and the second one contains only the green ones. 

We know that the width of the original image is 800 pixels (the array has 800 columns), so we can simply take a subset of the columns:

In [ ]:
# take columns with indices from 0 to 399 (first 400 columns)
red_apples = img_arr[:, 0:400, :]

(img_arr.shape, red_apples.shape)

In [ ]:

# take columns with indices from 400 to 799 (last 400 columns)
green_apples = img_arr[:, 400:800]

(img_arr.shape, green_apples.shape)

In [ ]:
# show the cropped images
import matplotlib.pyplot as plt

plt.subplot(1, 2, 1)
plt.imshow(red_apples)
plt.title("Red")

plt.subplot(1, 2, 2)
plt.imshow(green_apples)
plt.title("Green")

It works, but we can see some red apples among the green ones. Let's crop the image starting from column 500 instead, plus skip the first 150 rows. Let's do the same also for the red apples in order to let both images having the same size:

In [ ]:
# take rows with indices from 150 to 599 and columns with indices from 0 to 299 (first 300 columns)
red_apples = img_arr[150:600, 0:300]
red_apples.shape

In [ ]:
# take columns with indices from 500 to 799 (last 300) and the same rows
green_apples = img_arr[150:600, 500:800]
green_apples.shape

In [ ]:
# show the cropped images
plt.subplot(1, 2, 1)
plt.imshow(red_apples)
plt.title("Red")

plt.subplot(1, 2, 2)
plt.imshow(green_apples)
plt.title("Green")

Looks better! In fact, if an index involves the last row or the last column we can skip it and leave its place empty. Same in case if an index starts with the first (zero) row or column:

In [ ]:
# same operation but without explicit declaration of the last row and the first column
red_apples = img_arr[150:, :300]

# same operation but without explicit declaration of the last row and the last column
green_apples = img_arr[150:, 500:]

# show the cropped images
plt.subplot(1, 2, 1)
plt.imshow(red_apples)
plt.title("Red")

plt.subplot(1, 2, 2)
plt.imshow(green_apples)
plt.title("Green")

Now we can also *flip* the images vertically or horizontally (or both) by changing order of the rows and columns. In this case we need to provide an extra element in the sequence of indices, which will be the step. If we use step -1 we take the last row and make it first, then take the row before the last and make it the second etc. 

Here is how it works with rows:

In [ ]:
# flip image vertically by reordering rows of pixels in reverse order
red_apples_vf = red_apples[::-1, :]

# show the cropped images
plt.subplot(1, 2, 1)
plt.imshow(red_apples)
plt.title("Red (original)")

plt.subplot(1, 2, 2)
plt.imshow(red_apples_vf)
plt.title("Red (flipped vertically)")

Write a code which will also flip red apples vertically and both vertically and horizontally. Show a figure with all four images together — the original one and the three flipped ones (vertical, horizontal and both).

In [ ]:
# place your code here




If two (or more) images are represented by arrays with the same number of rows, columns or both, they can be merged into a new image. You can merge arrays (and hence the corresponding images) by stacking the vertically or horizontally. Here is an example:

In [ ]:
more_apples = np.hstack((red_apples, red_apples, red_apples))
plt.imshow(more_apples)

You can make a sequence of stacks. 

In [ ]:
even_more_apples = np.vstack((more_apples, more_apples))
plt.imshow(even_more_apples)

Here is another example:

In [ ]:
# make the crops again
red_apples = img_arr[150:, :300]
green_apples = img_arr[150:, 500:]

# flip the images both vertically and horizontally
red_apples_flipped = red_apples[::-1, ::-1]
green_apples_flipped = green_apples[::-1, ::-1]

# stack the images horizontally
img_top = np.hstack((red_apples, green_apples_flipped))
img_bottom = np.hstack((green_apples, red_apples_flipped))

# stack the previous results vertically
img = np.vstack((img_top, img_bottom))

# show the result
plt.imshow(img)

# show the dimension of the new image
img.shape

Finally you can also transpose an image — flipping it along its main diagonal. Here is illustration from [Wikipidea](https://en.wikipedia.org/wiki/Transpose):

<img src="illustrations/Matrix_transpose.gif" style="width:200px" >

Transposition can be also made by rotation of the image by 90 degrees and reversing the columns. In other words it simply makes rows as columns and columns as rows. 

Because we have 3D array (with three color channels) we need to tell NumPy explicitly how to transpose it. Because rows is the first dimension (with index 0), columns is the second dimension (with index 1) and color channels is the third dimension (with index 2), we define the new transposition of rows and columns by swapping their indices:

In [ ]:
# transpose image array
img_transposed = img.transpose((1, 0, 2))
img_transposed.shape

In [ ]:
# show the result
plt.imshow(img_transposed)

Try to change the code above to crop the image for focusing on the green apples instead.

**The transformation is not the same as rotation. Think what you need to combine the transposition with in order to rotate the image by 90, -90 or 180 degrees?** Try your ideas here:

In [ ]:
# place your code here



## Unfolding, refolding and scatter plots

One can think about a color image as a dataset, where pixels are samples (objects) and color channels are variables (columns) as it is shown below:

<img src="illustrations/image-as-data.png" style="width:700px" >

The operation of transforming the color image represented as 3D array to 2D data matrix is called *unfolding*. The opposite operation is usually called refolding.

Let's load the *Apples* image, unfold it and make a scatter plot where each pixel will be a point in cartesian space. Let's make such plot for red and green color channels:

In [ ]:
# load image and convert to NumPy array
img = Image.open("datasets/Apples.jpg")
img_arr = np.array(img)

img_arr.shape

As you can see, the image contains almost half of a million pixels, which will be quite heavy to process. Let's downsample the image by taking every fourth row and every fourth column:

In [ ]:
img_arr_small = img_arr[::4, ::4, :]
img_arr_small.shape

This should be doable, let's unfold first:

In [ ]:
# let's get every dimension to separate variable
height, width, nchannels = img_arr_small.shape

# compute total number of pixels
npixels = height * width

# unfold array to dataset
img_data = img_arr_small.reshape((npixels, nchannels))
img_data.shape

In [ ]:
# let's look at top rows
img_data[0:5, :]

In [ ]:
# make plot for red (1st channel) vs green (2nd channel)
plt.scatter(img_data[:, 0], img_data[:, 1], marker = ".")
plt.xlabel("Red")
plt.xlabel("Green")

 We can use color of pixels to colorize the points (will be a bit slower!):

In [ ]:
plt.scatter(img_data[:, 0], img_data[:, 1], marker = ".", edgecolors=None, color = img_data/255)
plt.xlabel("Red")
plt.ylabel("Green")

Another possibility is to compute density for every point on the plot — how many pixels are around. In this case we need to install another useful package, `scipy`, which contains a lot of useful functions.

In [ ]:
! pip install scipy

Now let's compute the density and use it for color coding (it will take some time!):

In [ ]:
from scipy.stats import gaussian_kde

# select only relevant columns and transpose selection
dens_data = img_data[:, 0:2].T

# calculate the point densities
dens = gaussian_kde(dens_data)(dens_data)

In [ ]:
# make a plot using plot densities for color gradient
plt.scatter(img_data[:, 0], img_data[:, 1], marker = ".", edgecolors=None, c = dens)
plt.xlabel("Red")
plt.ylabel("Green")
plt.colorbar()

By the way, the computed density values can be refolded to one-channel image:

In [ ]:
dens_img = dens.reshape((height, width))
plt.imshow(dens_img)

We can even make Principal Component Analysis (via Singular Value Decomposition) and show scores as images:

In [ ]:
# import SVD method from numpy
from numpy.linalg import svd

# mean center the columns
img_data_centered = img_data - img_data.mean(axis = 0)


In [ ]:

# computed SVD
U, S, V = svd(img_data_centered, full_matrices=False)

# compute scores
T = np.dot(U, np.diag(S))

In [ ]:

# refold matrix scores into image
score_img = T.reshape((height, width, nchannels))

# show the scores for PC1
plt.imshow(score_img[:, :, 0])
plt.title("PC1")
plt.colorbar()

In [ ]:
# show the scores for PC1 with thresholding
plt.imshow(score_img[:, :, 0] < 0)
plt.title("PC1")
plt.colorbar()

And of course you can use the result of thresholding as a mask and combine it with the original image. However in this case you need to use transposed arrays to let NumPy do multiplication of 3D array and 2D array correctly.

In [ ]:
mask = (score_img[:, :, 0] < 0)
img_masked = img_arr_small.T * mask.T

# show the scores for PC1 with thresholding
plt.imshow(img_masked.T)
plt.title("PC1")
plt.colorbar()


## Image transformations in PIL

Now let's learn how to do transformations in PIL. All transformations can be actually done manually by using NumPy arrays, but it will require a lot of code. So we will reuse the code already written by the PIL/Pillow developers. First of all you can do all operations we discussed above in PIL as well. 


### Cropping and flipping

Let's load the apple image again and crop it to focus on the part with red apples.

In [ ]:
img = Image.open("datasets/Apples.jpg")

# defining the bounding box (left, upper, right, lower) for red apples part
box_red = (0, 0, 300, 600)

# crop the image into new ones)
img_red = img.crop(box_red)

# show the cropped image
img_red

You can also flip images in PIL as shown the code below:

In [ ]:
img_flipped = img.transpose(Image.FLIP_LEFT_RIGHT)

img_flipped

### Resizing

Here is how to resize the image.

In [ ]:
# definig the new size in pixels (width, height)
new_size = (400, 300)

resized_img = img.resize(new_size)

resized_img

Keep in mind that resizing does not keep the correct aspect ratio, it is your responsibility. Otherwise you can make some distortions like the following.

In [ ]:
# definig the new size which does not keep the correct aspect ratio
new_size = (400, 100)

resized_img = img.resize(new_size)

resized_img

### Rotation

Rotation alters the orientation of an image by a specified angle, allowing for adjustments to the image's orientation or alignment. It's useful for correcting skewed images or achieving desired visual effects. 

In [ ]:
# Rotation the image by 40 degrees clockwise
rotated_image = img.rotate(40)

rotated_image

As you can see the rotated image has the same size as the original. You can change this if you add additional argument to the `rotate()` function:

In [ ]:
# Rotation the image by 40 degrees clockwise and resize
rotated_image = img.rotate(40, expand=1)

rotated_image

### Interpolation

Any transformation which changes the position of pixels requires additional operation — *interpolation*. The idea is as follows. The image is always a rectangle with fixed number of pixels. But when you e.g. rotate the pixels, they do not match the grid:

<img src="illustrations/interp1.png" width="400px">

Here is another illustration. 

<img src="illustrations/interp2.png" width="700px">

The black squares represent the grid of pixels on the final image. The color rectangles represent the old pixels after rotation. As you can see they do not match. So what we need to do is to compute intensities of the black pixels by taking into account the intensities of the colored pixels around.

Let's show the same but making the old and the new pixels round and more sparse for the sake of clarity:

<img src="illustrations/interp3.png" width="400px">

And our goal is to compute the intensity of the bold black pixel in the middle. There are three main possibilities (other ways also exist but will not be considered here):

**Nearest neighbor**

Simply takes the intensity of the closest pixel:

<img src="illustrations/interp-nn.png" width="400px">

**Bilinear interpolation**

Takes four pixels around:

<img src="illustrations/interp-bl.png" width="400px">

Fits their intensities using a set of linear functions and then computes the new intensity based on the interpolations:

<img src="illustrations/interp-bl2.png" width="700px">

**Bicubic interpolation**

Takes 16 pixels around:

<img src="illustrations/interp-bc.png" width="400px">

Fits their intensities with a set of cubic polynomials and then computes the new intensity based on the functions (not shown here).

The simplest and the fastest is *nearest neighbor* method and this is also the default ones. If you want to use other methods, simply specify additional parameter to the transformation function as it is shown in the example below:
 

In [ ]:
# Rotation the image by 40 degrees clockwise and resize using bilinear interpolation
rotated_image = img.rotate(40, expand=1, resample=Image.Resampling.BILINEAR)

rotated_image

For high resolution image with many pixels the difference is barely visible by a naked eye but for small images it can be crucial. See an example below:

In [ ]:
# take a small crop of the original image
img_small = img.crop([100, 100, 180, 180])
img_small.size

In [ ]:
img_small

In [ ]:
# increase size of the cropped image using three different methods
img_resized_nn = img_small.resize((400, 400), resample=Image.Resampling.NEAREST)
img_resized_bl = img_small.resize((400, 400), resample=Image.Resampling.BILINEAR)
img_resized_bc = img_small.resize((400, 400), resample=Image.Resampling.BICUBIC)

In [ ]:
# investigate the images one by one
img_resized_nn

### Shearing and other transformations

You can also distort the images by applying geometric transformation which changes the coordinates of the pixels so they do not match the original coordinate grid. One of the examples of such distortion is *[shearing](https://en.wikipedia.org/wiki/Shear_mapping)*.

Shearing distorts the shape of an image along one axis by shifting each point by an amount proportional to its distance from a particular axis. This transformation introduces a "sheared" appearance, useful for creating unique visual effects or correcting perspective distortions.

In [ ]:
# define the shearing factor
shear_factor = 0.1

# shearing the image horizontally
sheared_image = img.transform(img.size, Image.AFFINE, (1, shear_factor, 0, 0, 1, 0))

sheared_image

The method `transform` can apply any geometric transformation to the pixels. Second argument of the method, `Image.AFFINE`, "tells" that the transformation should be [affine](https://en.wikipedia.org/wiki/Affine_transformation). Affine transformation computes new coordinates of every pixel $(x', y')$ as a linear combination of its old coordinates $(x, y)$:

$x' = ax + by + c$

$y' = dx + ey + f$

So to apply a transformation, you simply need to provide all six coefficients in form of a tuple: `(a, b, c, d, e, f)`. For example, coefficients `(1, 0, 0, 0, 1, 0)` does not change any coordinates, this is an identity transformation.

And what we did above we applied the following transformation:

$x' = x + sy$

$y' = y$

Where $s$ was equal to 0.1. So new x-coordinate depends on the y-coordinate of the pixels, while y-coordinates do not change. Because y-coordinates of image pixels start from the top and increase to the bottom, we see a bigger shift for the bottom pixels. The larger shearing factor will be the larger shift.

Now we can implement the vertical sheering:

$x' = x$

$y' = sx + y$



In [ ]:
# define the shearing factor
shear_factor = 0.1

# shearing the image vertically
sheared_image = img.transform(img.size, Image.AFFINE, (1, 0, 0, shear_factor, 1, 0))

sheared_image

And in both directions:

In [ ]:
# define the shearing factor
shear_factor = 0.1

# shearing the image vertically
sheared_image = img.transform(img.size, Image.AFFINE, (1, shear_factor, 0, shear_factor, 1, 0))

sheared_image

Try to play with the sheering factor and see how it affects the outcome. 

You probably wonder why distort the images? Well in fact these kind of transformations are mostly used to do the opposite — remove the distortion introduced e.g. by camera or other obstacles. But sometimes distortion is also a way to enrich the image dataset in order to train a good model, which we will discuss in the last class.